In [6]:
import numpy as np

import sys
sys.path.insert(0,'..')

import souploader

import pandas as pd

In [7]:
from collections import defaultdict

In [8]:
import srs

import bballutils

In [9]:
num_games = 63
missing = {
    'jt': [11,12,13,14,15,43,62],
    'jb': [21,22,34,47,57,59,60],
    'ms': [8] + list(range(19,37)) + [58,63],
    'kw': list(range(1,12)) + [16,20,25,28,32,40,43,46,52,58,60,62,63],
    'rw': [2,10,11,12,13,24,25,48] + list(range(56,63)),
#     'tt': [5,10] + list(range(39,52)),
#     'pp': list(range(15,21)),
}

In [10]:
all_games = np.full(num_games, len(missing))

for i in missing.values():
    for j in i:
        game = j-1
        
        all_games[game] -= 1
all_games

array([4, 3, 4, 4, 4, 4, 4, 3, 4, 3, 2, 3, 3, 4, 4, 4, 5, 5, 4, 3, 3, 3,
       4, 3, 2, 4, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 5, 5, 5, 4, 5, 5, 3, 5,
       5, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 3, 2, 3, 2, 4, 2, 3])

# load game results

In [11]:
# Load just boston games
soup = souploader.load_soup('https://www.basketball-reference.com/teams/BOS/2021_games.html')

table = souploader.soup_get_table_as_pandas(soup, id='games')

table = table[table['Opponent'] != 'Opponent']

table = table.rename(columns={'Unnamed: 7': 'Result'})

table['G'] = pd.to_numeric(table['G'])

table['Tm'] = pd.to_numeric(table['Tm'])
table['Opp'] = pd.to_numeric(table['Opp'])

In [12]:
all_games_temp = np.full(len(table), np.nan)
all_games_temp[:len(all_games)] = all_games
table['NumStarters'] = all_games_temp

In [13]:
n_starters = 5
print(bballutils.pandas2md(
    table[table['NumStarters'] >= n_starters]
        [['Result','Opponent','Date','Tm','Opp']]
        .astype({'Tm': 'int32','Opp': 'int32'})
))

Result|Opponent|Date|Tm|Opp
----|----|----|----|----
L|San Antonio Spurs|Wed, Jan 27, 2021|106|110
L|Los Angeles Lakers|Sat, Jan 30, 2021|95|96
L|Brooklyn Nets|Thu, Mar 11, 2021|109|121
W|Houston Rockets|Sun, Mar 14, 2021|134|107
L|Utah Jazz|Tue, Mar 16, 2021|109|117
L|Sacramento Kings|Fri, Mar 19, 2021|96|107
W|Orlando Magic|Sun, Mar 21, 2021|112|96
L|Milwaukee Bucks|Wed, Mar 24, 2021|119|121
W|Milwaukee Bucks|Fri, Mar 26, 2021|122|114
W|Houston Rockets|Fri, Apr 2, 2021|118|102
W|Charlotte Hornets|Sun, Apr 4, 2021|116|86
L|Philadelphia 76ers|Tue, Apr 6, 2021|96|106
W|Minnesota Timberwolves|Fri, Apr 9, 2021|145|136
W|Denver Nuggets|Sun, Apr 11, 2021|105|87
W|Portland Trail Blazers|Tue, Apr 13, 2021|116|115


In [14]:
results = np.empty(len(table))
differential = np.empty(len(table))
for i,j in table.iterrows():
    game = int(j['G'])-1
    results[game] = 1 if j['Result'] == 'W' else 0
    differential[game] = j['Tm'] - j['Opp']

In [15]:
print('total wins = {:.0f}'.format(results.sum()))

total wins = 33


In [16]:
win_by_start = np.zeros(6)
games_by_start = np.zeros(6)
diff_by_start = np.zeros(6)

for idx, (nstarters, win, diff) in enumerate(zip(all_games, results, differential), start=1):
    win_by_start[nstarters] += win
    games_by_start[nstarters] += 1
    diff_by_start[nstarters] += diff
    if nstarters == 2 and win > 0:
        print(table.loc[table['G'] == idx,['G','Date','Opponent','Result','Tm','Opp']])

# win_by_start[4] += win_by_start[5]
# games_by_start[4] += games_by_start[5]
# diff_by_start[4] += diff_by_start[5]

     G               Date       Opponent Result     Tm   Opp
10  11  Fri, Jan 15, 2021  Orlando Magic      W  124.0  97.0


In [17]:
for i, (w,g,d) in enumerate( zip(win_by_start, games_by_start, diff_by_start) ):
    l = g-w
    if g > 0:
        print('{}: {}-{} {:.03f} {:.01f}'.format(i,int(w),int(l),w/g,d/g))

2: 1-4 0.200 1.2
3: 8-8 0.500 -2.6
4: 16-11 0.593 2.5
5: 8-7 0.533 5.1


# SRS

In [18]:
# Load all games
#
# Note the months list!!
#
games_df = srs.load_games(2021, ['december','january','february','march','april'])

In [19]:
# split_match = all_games <= 4
split_match = all_games <= 3
# split_match = all_games <= 2
split_games = table[:len(all_games)][split_match]['G'].values
split_games

array([ 2,  8, 10, 11, 12, 13, 20, 21, 22, 24, 25, 28, 32, 34, 43, 57, 58,
       59, 60, 62, 63])

In [20]:
hosp_games_df = srs.split_team(games_df, srs.BOS, split_games, 'Hospital Celtics')

a = srs.get_srs(hosp_games_df)
a['rank'] = np.arange(len(a))+1

b = srs.get_srs(games_df)
b['rank'] = np.arange(len(b))+1

srs_df = a.join(b, how='outer', lsuffix='_hospital').sort_values(by='srs_hospital',ascending=False)

srs_df.style.format({'srs_hospital': '{:.2f}', 
                     'srs':'{:.2f}',
                     'rank_hospital': '{:.0f}',
                     'rank': '{:.0f}'})

,srs_hospital,rank_hospital,srs,rank
Utah Jazz,9.50,1,9.45,1
Milwaukee Bucks,6.54,2,6.47,2
Los Angeles Clippers,6.10,3,6.12,3
Phoenix Suns,5.64,4,5.75,4
Philadelphia 76ers,5.22,5,5.23,5
Denver Nuggets,4.95,6,4.89,6
Brooklyn Nets,4.09,7,4.19,7
Boston Celtics,3.68,8,1.93,11
Los Angeles Lakers,3.11,9,3.06,8
Dallas Mavericks,2.43,10,2.37,9
